In [10]:
import requests
import json

In [2]:
datawolf_base = "https://incore-dev.ncsa.illinois.edu/datawolf/"
datwolf_workflow_uri = datawolf_base + "workflows/"

In [3]:
def get_workflow_def(datwolf_workflow_uri, workflow_id,token):
    res = requests.get(datwolf_workflow_uri+workflow_id , headers={"authorization": token})
    workflow_def = res.json()
    
    return workflow_def

In [4]:
# template
def construct_datwolf_config_template(workflow_def):
    return {
        "title": workflow_def["title"],
        "description": workflow_def["description"],
        "wfTitle": workflow_def["title"],
        "wfId": workflow_def["id"],
        "creatorId": workflow_def["creator"]["id"], 
        "inputs": {
            "parameters": [],
            "datasets": []
        },
        "outputs": {
            "datasets":[{
                "title": "",
                "description": "",
                "datasetsMap": [],
            }]
        },
        "metadata":[]
    }

In [5]:
def construct_datwolf_config_parameters(workflow_def, datawolf_config_template):
    parameters = {}
    for step in workflow_def["steps"]:
        for param in step["tool"]["parameters"]:
            if param["id"] not in parameters.keys():
                parameters[param["id"]] = {
                    "title": param["title"],
                    "description": param["description"],
                    "variableName":"",
                    "widget":None,
                    "hidden":False,
                    "defaultValue":param["value"],
                    "parametersMap":[{
                        "paramId": step["parameters"][param["parameterId"]],
                        "parameters": {
                            "step": step["title"],
                            "title": param["title"]
                        }
                    }]
                }
            else:
                # append to parametersMap
                parameters[param["id"]]["parametersMap"].append({
                    "paramId": step["parameters"][param["parameterId"]],
                        "parameters": {
                            "step": step["title"],
                            "title": param["title"]
                        }
                })
    for entry in parameters.values():
        datawolf_config["inputs"]["parameters"].append(entry)
    
    return datawolf_config_template

In [6]:
def construct_datawolf_config_input_datasets(workflow_def, datawolf_config_template):
    datasets = {}
    for step in workflow_def["steps"]:
        for dt in step["tool"]["inputs"]:
            if dt["id"] not in datasets.keys():
                datasets[dt["id"]] = {
                        "title": dt["title"],
                        "description": dt["description"],
                        "variableName":"",
                        "widget":None,
                        "hidden":True,
                        "datasetsMap":[
                            {
                                "datasetId": step["inputs"][dt["dataId"]],
                                "datasets": {
                                    "step": step["title"],
                                    "title": dt["title"]
                            }
                        }]
                    }
            
            else:
                # append to parametersMap
                datasets[dt["id"]]["datasetsMap"].append({
                                "datasetId": step["inputs"][dt["dataId"]],
                                "datasets": {
                                    "step": step["title"],
                                    "title": dt["title"]
                                }
                            })
    for entry in datasets.values():
        datawolf_config["inputs"]["datasets"].append(entry)
    
    return datawolf_config_template

In [19]:
def construct_datawolf_config_output_datasets(workflow_def, datawolf_config_template):
    
    # outputs are unique
    for step in workflow_def["steps"]:
        for dt in step["tool"]["outputs"]:
            if dt["title"] == "stdout":
                widget = "Log"
            else:
                widget = ""
            datawolf_config["outputs"]["datasets"][0]["datasetsMap"].append({
                "datasetId":step["outputs"][dt["dataId"]],
                "datasets": {
                        "step": step["title"],
                        "title": dt["title"],
                        "variableName": (step["title"]+dt["title"]).replace(" ", ""),
                        "widget": widget
                    }
            })     
    
    return datawolf_config_template

In [20]:
workflow_id = "05e5ac18-c202-4a8b-973d-01afcae20b14"
token = "bearer eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJabHVyQldkLUhHV2ZxeUNkb28yc1FEbmd4RE9mbEVZeks3My05UXpnY0JBIn0.eyJleHAiOjE2NzgxNjQ1NDMsImlhdCI6MTY3ODEyMTM0MywianRpIjoiYTA1YmViZjEtNDQ3MC00Mzk2LTgzZTItODJkZjRlNzc4OTdjIiwiaXNzIjoiaHR0cHM6Ly9pbmNvcmUtZGV2Lm5jc2EuaWxsaW5vaXMuZWR1L3JlYWxtcy9Jbi1jb3JlIiwiYXVkIjpbIklOQ09SRSIsImFjY291bnQiXSwic3ViIjoiZDNiZGUyMmYtMGY5Ni00NGQ3LTgyMjktMDFkMTNhYWM4ZGZmIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoicmVhY3QtYXV0aCIsInNlc3Npb25fc3RhdGUiOiI0NjZmNGI4Zi1hZTNhLTQyMjQtYjEwNS03NTRjYTdiYmIxNTgiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHA6Ly9sb2NhbGhvc3Q6MzAwMCJdLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiIsImluY29yZV91c2VyX3JvbGUiXX0sInJlc291cmNlX2FjY2VzcyI6eyJhY2NvdW50Ijp7InJvbGVzIjpbIm1hbmFnZS1hY2NvdW50Iiwidmlldy1wcm9maWxlIl19fSwic2NvcGUiOiJwcm9maWxlIGVtYWlsIiwic2lkIjoiNDY2ZjRiOGYtYWUzYS00MjI0LWIxMDUtNzU0Y2E3YmJiMTU4IiwiZW1haWxfdmVyaWZpZWQiOnRydWUsInVpZF9udW1iZXIiOjU2MDIxLCJuYW1lIjoiQ2hlbiBXYW5nIiwiZ3JvdXBzIjpbImluY29yZV9qdXB5dGVyIiwiaW5jb3JlX3NsY191c2VyIiwiaW5jb3JlX2dhbHZlc3Rvbl91c2VyIiwiaW5jb3JlX2pvcGxpbl91c2VyIiwiaW5jb3JlX3VzZXIiLCJpbmNvcmVfY29lIiwiaW5jb3JlX25jc2EiXSwicHJlZmVycmVkX3VzZXJuYW1lIjoiY3dhbmcxMzgiLCJnaXZlbl9uYW1lIjoiQ2hlbiIsImZhbWlseV9uYW1lIjoiV2FuZyIsImVtYWlsIjoiY3dhbmcxMzhAaWxsaW5vaXMuZWR1In0.Jm9R-gNhp45Zzz9Z1SQAMoupYIY7uIe5iDjTMyDJR4H3GFZAPEteu9JKsY4PUwTwIwP5-i7f6t2mVQbL6EsRUJaV-cUTvyN13c0WBTo4EJpPPE1cYkk2cMAJLMkdU5GOashhXebszG9gf8eE-3eRd2GfFtGzF8W8neH9x7kofz3xsmiDblDLR65Cre03LrcowcWGlOgveQCmIKThQ9nEYa4SPuqKML8mMbiMSCtD7CyB873n6PumfuKeW99SdEDLT0NnoDIKT6MYXJ2VKhqXodDHpcu2aIYlcjwXbzeu3UoFAeRxd0zg-qzf0RDScq9ATe18Zk_6sX3ogIpomogw8w"

workflow_def = get_workflow_def(datwolf_workflow_uri, workflow_id,token)
datawolf_config = construct_datwolf_config_template(workflow_def)
datawolf_config = construct_datwolf_config_parameters(workflow_def, datawolf_config)
datawolf_config = construct_datawolf_config_input_datasets(workflow_def, datawolf_config)
datawolf_config = construct_datawolf_config_output_datasets(workflow_def, datawolf_config)

In [21]:
# save as json
with open("datawolf.config.json", "w") as f:
    json.dump(datawolf_config, f, indent=2)